# Baseline ELECTRA Training

This notebook demonstrates training a baseline ELECTRA model on NLP benchmarks.

## Setup

In [ ]:
import sys
sys.path.append('..')

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Load Dataset

We'll use SNLI for this example.

In [ ]:
# Load SNLI dataset
dataset = load_dataset('snli')

# Filter out invalid labels (-1)
train_dataset = dataset['train'].filter(lambda x: x['label'] != -1)
val_dataset = dataset['validation'].filter(lambda x: x['label'] != -1)

# For quick testing, use a subset
train_dataset = train_dataset.select(range(1000))
val_dataset = val_dataset.select(range(500))

print(f"Train examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")
print(f"\nExample: {train_dataset[0]}")

## Load Model and Tokenizer

In [ ]:
model_name = 'google/electra-small-discriminator'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # entailment, neutral, contradiction
)

print(f"Model loaded: {model_name}")
print(f"Number of parameters: {model.num_parameters():,}")

## Preprocess Data

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("Preprocessing complete!")

## Training

In [ ]:
import evaluate
import numpy as np

# Load accuracy metric
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='../models/baseline_snli',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train!
trainer.train()

## Evaluation

In [ ]:
# Evaluate on validation set
eval_results = trainer.evaluate()
print(f"\nValidation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value}")

## Save Model

In [ ]:
# Save the model
trainer.save_model('../models/baseline_snli')
print("Model saved!")